## Парсер на основе библиотеки Scrapy

In [1]:
import platform
import scrapy
import logging
from scrapy.crawler import CrawlerProcess
import pandas as pd
from w3lib.html import strip_html5_whitespace as shw
from IPython.core.interactiveshell import InteractiveShell
import warnings
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

### Настройка пайплайнов
Класс `JsonWriterPipeline` создает пайплайн, в который пишет все найденные элементы в формат `JSON`.
Все данные будут сохраняться в два файл: первый с расширением `.json`, второй с расширением `.jl` (в нем каждый элемент json находится на новой странице).

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('data/wildberries/wildberries_data.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

#### Список предметов, которые нужно искать

### Создание класса для парсинга
В списке `start_urls` генерируются URL страниц, из которых нужно извлечь информацию: 
- `name` - наименование продукта
- `current_price` - текущая цена
- `old_price` - старая цена
- `img_url` - ссылка на фотографию

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'file:/Users/alexandrasorokovikova/2 курс/Формальные языки/Проект парсеры/parsing-with-python/tests/wildberries/test_source/test{0}.html'.format(i) for i in range(0, 52)
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',                               
        'FEED_URI': 'data/wildberries/wildberries_data.json'                      
    }
    
    def parse(self, response):
        prices = response.xpath("//span[@class='price-commission__current-price']/text()").get()
        yield {
            'name': response.xpath("//span[@class='goods-name']/text()").get(),
            'current_price': response.xpath("//span[@class='price-commission__current-price']/text()").get(),
            'old_price': shw(response.xpath("//del[@class='price-commission__old-price']/text()").get()),
            'img_url': response.xpath("//div[@class='product-card__img-wrap']/img/@src").get(),
        }
        

### Парсинг

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2021-10-19 00:16:18 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-19 00:16:18 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-20.2.0-x86_64-i386-64bit
2021-10-19 00:16:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-19 00:16:18 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


### Обзор полученных данных

In [5]:
wildberries_dataset = pd.read_json('data/wildberries/wildberries_data.json')
wildberries_dataset.head(10)

,name,current_price,old_price,img_url
0,"Да, ваш ребенок - гений!",1 553 ₽,2 560 ₽,//images.wbstatic.net/c324x432/new/17040000/17...
1,Смартфон,20 186 ₽,21 990 ₽,//images.wbstatic.net/c324x432/new/30130000/30...
2,Крем для лица с Алоэ Вера/питает кожу/ увлажня...,845 ₽,1 700 ₽,//images.wbstatic.net/c324x432/new/41480000/41...
3,Настольная карточная игра Хугермугер Сиквел,562 ₽,1 000 ₽,//images.wbstatic.net/c324x432/new/7530000/753...
4,"Телевизор UE55TU7090UXRU / 55"" / 4K UHD / Blue...",47 675 ₽,54 990 ₽,//images.wbstatic.net/c324x432/new/14880000/14...
5,Ноутбук AMD Ryzen 3 4300U / 8 ГБ / 512 Гб / 15...,47 818 ₽,52 090 ₽,//images.wbstatic.net/c324x432/new/34210000/34...
6,Постельное белье комплект КПБ жатый Сатин хлоп...,9 496 ₽,14 000 ₽,//images.wbstatic.net/c324x432/new/29990000/29...
7,Платье /Платье женское повседневное /Офисное ж...,2 077 ₽,8 400 ₽,//images.wbstatic.net/c324x432/new/25170000/25...
8,Ботинки женские / Осенние/ Обувь женская/Ботин...,3 405 ₽,9 540 ₽,//images.wbstatic.net/c324x432/new/42340000/42...
9,Фотообои/Фотообои на стену/Фотообои флизелинов...,2 864 ₽,5 731 ₽,//images.wbstatic.net/c324x432/new/37930000/37...


In [6]:
wildberries_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           52 non-null     object
 1   current_price  52 non-null     object
 2   old_price      52 non-null     object
 3   img_url        52 non-null     object
dtypes: object(4)
memory usage: 1.8+ KB


Получилось 52 непустых объекта